This notebooks serves as a demo of `TorchModel` capabilities.

Aside from that, it is also used as an interactive test.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn


sys.path.insert(0, "../../..")
from batchflow import *
from batchflow.opensets import MNIST
from batchflow.models.torch import *
from batchflow.models.torch.layers import *
from batchflow.models.torch.callbacks import ReduceLROnPlateau, EarlyStopping

# Setup: global parameters and functions

In [ ]:
# Global parameters
mnist = MNIST(bar=False)

BAR = True
PLOT = False

IMAGE_SHAPE = (1, 28, 28)
BATCH_SIZE = 16
N_ITERS = 10
N_ITERS_LARGE = 100

if __name__ == '__main__':
    MICROBATCH = None
    DEVICE = 'gpu:0'
    BAR = 't'
    PLOT = True

In [ ]:
def get_classification_config(model_class, config):
    default_config = {
        # Shapes info. Can be commented
        'inputs_shapes': IMAGE_SHAPE,
        'classes': 10,

        'loss': 'ce',
        'microbatch_size': MICROBATCH,
        'device': DEVICE,
    }

    if 'inputs_shapes' in config and isinstance(config['inputs_shapes'], list):
        inputs = [B.images for item in config['inputs_shapes']]
    else:
        inputs = B.images
    
    
    pipeline_config = {
        'model': model_class,
        'model_config': {**default_config, **config},
        'inputs': inputs,
        'targets': B.labels,

        'gather': {'metrics_class' : 'classification',
                   'fmt' : 'logits',
                   'axis' : 1,
                   'targets' : B.labels},
        'evaluate': 'accuracy',
    }
    return pipeline_config

def get_segmentation_config(model_class, config):
    default_config = {
        # Shapes info. Can be commented
        'inputs_shapes': IMAGE_SHAPE,
        'targets_shapes': IMAGE_SHAPE,

        'loss': 'mse',
        'microbatch': MICROBATCH,
        'device': DEVICE,
    }
    
    if 'inputs_shapes' in config and isinstance(config['inputs_shapes'], list):
        inputs = [B.images for item in config['inputs_shapes']]
    else:
        inputs = B.images
    
    pipeline_config = {
        'model': model_class,
        'model_config': {**default_config, **config},
        'inputs': inputs,
        'targets': B.images,

        'gather': {'metrics_class' : 'segmentation',
                   'fmt' : 'proba',
                   'axis' : None,
                   'targets' : B.images},
        'evaluate': 'jaccard',
    }
    return pipeline_config

In [ ]:
def get_pipeline(pipeline_config):
    """ Pipeline config must contain 'model', 'model_config', 'feed_dict' keys. """
    pipeline = (Pipeline(config=pipeline_config)
                .init_variable('loss_history', [])
                .to_array(channels='first', dtype='float32')
                .multiply(multiplier=1/255., preserve_type=False)
                .init_model(name='MODEL', model_class=C('model'), config=C('model_config'))
                .train_model('MODEL',
                             inputs=pipeline_config['inputs'],
                             targets=pipeline_config['targets'],
                             outputs='loss',
                             save_to=V('loss_history', mode='a'))
                )
    return pipeline

In [ ]:
def run(task, model_class, config, description, batch_size=BATCH_SIZE, n_iters=N_ITERS, **kwargs):
    if task == 'classification':
        pipeline_config = get_classification_config(model_class, config)
    elif task == 'segmentation':
        pipeline_config = get_segmentation_config(model_class, config)

    train_pipeline = get_pipeline(pipeline_config) << mnist.train
    _ = train_pipeline.run(batch_size, n_iters=n_iters,
                           bar={'bar': BAR, 'monitors': 'loss_history'},
                           **kwargs)
    
    print(f'{task} "{description}" is done! Number of parameters in the model: {train_pipeline.model.num_parameters:,}')
    return train_pipeline

In [ ]:
def show_some_results(ppl, task, size=10):
    batch_ind = np.random.randint(len(ppl.v('targets')))
    image_ind = np.random.choice(len(ppl.v('targets')[batch_ind]), size=size, replace=False)
    true = ppl.v('targets')[batch_ind]
    pred = ppl.v('predictions')[batch_ind]

    if task == 'classification':
        print(pd.DataFrame({'true': true[image_ind],
                            'pred': np.argmax(pred[image_ind], axis=1)}).to_string(index=False))
    elif task == 'segmentation':
        pass # for the sake of parsing by notebooks_test.py
        fig, ax = plt.subplots(2, size, figsize=(10, 5))
        [axi.set_axis_off() for axi in ax.ravel()]
        for plot_num, image_num in enumerate(image_ind):
            ax[0][plot_num].imshow(true[image_num][0], cmap='gray', vmin=0, vmax=1)
            ax[1][plot_num].imshow(pred[image_num][0], cmap='gray', vmin=0, vmax=1)

In [ ]:
def test(pipeline, show_results=PLOT, batch_size=64, n_epochs=1, drop_last=False):
    test_pipeline = (mnist.test.p
                    .import_model('MODEL', pipeline)
                    .init_variable('targets', default=[])
                    .init_variable('predictions', default=[])
                    .init_variable('metrics', default=[]) 
                    .to_array(channels='first', dtype='float32')
                    .multiply(multiplier=1/255., preserve_type=False)
                    .update(V('targets', mode='a'), pipeline.config['targets'])
                    .predict_model('MODEL',
                                   inputs=pipeline.config['inputs'],
                                   outputs='predictions',
                                   save_to=V('predictions', mode='a'))
                    .gather_metrics(**pipeline.config['gather'], predictions=V.predictions[-1],
                                    save_to=V('metrics', mode='a'))
                    .run(batch_size, shuffle=False, n_epochs=n_epochs, drop_last=drop_last, bar=BAR)
    )
    
    if show_results:
        show_some_results(test_pipeline, pipeline.config['gather/metrics_class'])

    metrics = test_pipeline.get_variable('metrics')
    to_evaluate = pipeline.config['evaluate']
    evaluated = np.mean([m.evaluate(to_evaluate) for m in metrics])
    print(f'{to_evaluate} metrics is: {evaluated:.3}')

    return test_pipeline

# Tester
Test the simplest possible model and record timings

In [ ]:
config = {'initial_block': {'layout': 'Vf', 'features': 10}}
ppl = run('classification', TorchModel, config, 'simple fc', n_iters=2, batch_size=2)

In [ ]:
%%time
test(ppl, show_results=False, batch_size=BATCH_SIZE, drop_last=True, n_epochs=5);

# Classification

In [ ]:
config = {
    'initial_block': {'layout': 'fa'*2,
                      'features': [64, 128],},
    'body': {'layout': 'fa'*2,
             'features': [256, 512]},
    'head': {'layout': 'faf',
             'features': [600, 10]},
}


ppl = run('classification', TorchModel, config, 'simple fc', n_iters=200, batch_size=64)
test(ppl, show_results=False);

In [ ]:
config = {
    'body': {'type': 'encoder',
             'output_type': 'tensor',
             'num_stages': 3,
             'blocks/channels': '2 * same'},
    'head': {'layout': 'f'}
}

ppl = run('classification', TorchModel, config, 'encoder')

In [ ]:
# Example with multiple inputs
config = {
    'inputs_shapes': [IMAGE_SHAPE, IMAGE_SHAPE],

    'initial_block': {'type': 'wrapper',
                      'input_type': 'list',
                      'input_index': slice(None),
                      'module': Combine(op='concat', force_resize=False),},
    'body': {'type': 'encoder',
             'num_stages': 3,
             'output_type': 'tensor',
             'blocks/channels': '2 * same'},
    'head': {'layout': 'faf', 'features': [50, 10]}
}

ppl = run('classification', TorchModel, config, 'duo input')
ppl.model.repr(1)

# Classification: named networks

In [ ]:
ppl = run('classification', VGG16, {}, 'vgg16', n_iters=100, batch_size=128)
test(ppl, show_results=False);

In [ ]:
ppl = run('classification', ResNet18, {}, 'resnet18', n_iters=100, batch_size=128)
test(ppl, show_results=False);

In [ ]:
ppl = run('classification', SEResNeXt18, {}, 'SE-ResNeXt18', n_iters=100, batch_size=128)
test(ppl, show_results=False);

In [ ]:
ppl = run('classification', DenseNetS, {}, 'DenseNetS', n_iters=100, batch_size=128)
test(ppl, show_results=False);

In [ ]:
ppl = run('classification', EfficientNetB0, {}, 'EfficientNetB0', n_iters=100, batch_size=128)
test(ppl, show_results=False);

In [ ]:
config = {
    'initial_block': {'layout': 'cna', 'channels': 3,
                      'kernel_size': 5, 'stride': 1, 'padding': 'same'},
    'body': {'num_stages': 4,
             'blocks': {'n_reps': [1, 1, 2, 1],
                        'channels': '2 * same',
                        'attention': 'se'}
            }
}

ppl = run('classification', ResNet, config, 'resnet with config', n_iters=50, batch_size=128)
test(ppl, show_results=False);

In [ ]:
# reusing encoder from model from the previous cell
config = {
    'initial_block': {'type': 'wrapper', 'module': ppl.model.model.initial_block},
    'body': {'type': 'wrapper', 'module': ppl.model.model.body},
    'head' : {'type': 'wrapper', 'module': ppl.model.model.head},
}

ppl = run('classification', TorchModel, config, 'reused encoder', n_iters=50, batch_size=32)
test(ppl, show_results=False);

# Classification: imported models

In [ ]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Identity()

config = {
    'initial_block': {'layout': 'cna',
                      'channels': 3},
    'body': {'type': 'wrapper', 'module': resnet18},
    'head': {'layout': 'Dnfaf',
             'features': [50, 10],
             'dropout_rate': 0.3,
             'multisample': 0.3},
}

ppl = run('classification', TorchModel, config, 'torchvision resnet', n_iters=100, batch_size=128)
test(ppl, show_results=False);

In [ ]:
config = {
    'initial_block': {
        'layout': 'cna',
        'channels': 3,
        'output_list': True
    },
    'body': {
        'type': 'timm',
        'output_type': 'tensor',
        'path': 'resnet34d',
        'pretrained': True,
    },
    'head': {'layout': 'Dnfaf',
             'features': [50, 10],
             'dropout_rate': 0.3,
             'multisample': 0.3},
}

ppl = run('classification', TorchModel, config, 'TIMM-resnet34', n_iters=100, batch_size=128)
# ppl.model.model.body.config
test(ppl, show_results=False);

In [ ]:
config = {
    'trainable': ['initial_block', 'head'],
    **config
}

ppl = run('classification', TorchModel, config, 'TIMM-resnet34 finetune', n_iters=100, batch_size=128)
# ppl.model.model.body.config
test(ppl, show_results=False);

In [ ]:
config = {
    'initial_block': {
        'layout': 'cna',
        'channels': 3,
        'output_list': True
    },
    'body': {
        'type': 'hugging-face',
        'output_type': 'tensor',
        'path': 'facebook/convnext-tiny-224',
        'num_stages': 2,
        'depths': [2, 2],
        'num_channels': 3,        # number of input channels
        'patch_size': 2,          # ~stride in the beginning
        'hidden_sizes': [48, 64], # ~channels
    },
    'head': {'layout': 'Dnfaf',
             'features': [50, 10],
             'dropout_rate': 0.3,
             'multisample': 0.3},
}

ppl = run('classification', TorchModel, config, 'HF-ConvNext', n_iters=100, batch_size=128)
# ppl.model.model.body.config
test(ppl, show_results=False);

In [ ]:
config = {
    'initial_block': {
        'layout': 'cna',
        'channels': 3,
        'output_list': True
    },
    'body': {
        'type': 'hugging-face',
        'output_type': 'tensor',
        'path': 'nvidia/segformer-b0-finetuned-ade-512-512',
        'num_encoder_blocks': 2,                              # ~num_stages
        'strides': [2, 2],
        'num_attention_heads': [1, 4],
        'hidden_sizes': [48, 64],                             # ~channels
    },
    'head': {'layout': 'Dnfaf',
             'features': [50, 10],
             'dropout_rate': 0.3,
             'multisample': 0.3},
}

ppl = run('classification', TorchModel, config, 'HF-SegFormer', n_iters=100, batch_size=128)
# ppl.model.model.body.config
test(ppl, show_results=False);

In [ ]:
ppl.model.repr()

# Segmentation

In [ ]:
config = {
    'initial_block': {'layout': 'cna', 'channels': 1},
    'body': {'type': 'decoder',
             'num_stages': 3,
             'order': ['block'],}
}

ppl = run('segmentation', TorchModel, config, 'decoder')
ppl.model.repr()

In [ ]:
config = {
    'initial_block': {
        'layout': 'cnaRp cnaRp tna+ tna+ BScna+ cnac',
        'channels': [16, 32, 32, 16, 'same', 8, 1],
        'custom_padding': False,
        'transposed_conv': {'kernel_size': 2, 'stride': 2},
        'branch': {'layout': 'ca', 'channels': 'same'}
    },
}

ppl = run('segmentation', TorchModel, config, 'hardcoded unet')

In [ ]:
config = {
    'order': ['initial_block', 'encoder', 'embedding', 'decoder', 'head'],
    
    'initial_block': {
        'layout': 'cna',
        'channels': 8
    },
    'encoder': {
        'type': 'encoder',
        'num_stages': 3,
        'blocks/channels': '2 * same',
        'skip': {'channels': 'int(1.2 * same)', 'layout': 'cna'}
    },
    'embedding': {
        'layout': 'cna',
        'channels': 'same', 
        'input_type': 'list',
        'output_type': 'list',
    },
    'decoder': {
        'type': 'decoder',
        'blocks/channels': 'same // 2',
        'upsample': {'layout': 'b', 'factor': 2},
    },
    'head': {
        'layout': 'c', 
        'channels': 1}
}

ppl = run('segmentation', TorchModel, config, 'encoder->embedding->decoder from scratch')
ppl.model.model.repr(2)

In [ ]:
config = {
    'order': ['initial_block', 'encoder', 'embedding', 'decoder', 'head'],
    
    'encoder': {
        'type': 'encoder',
        'num_stages': 2,
        'blocks/channels': '4 * same'
    },
    'embedding': {
        'base_block': ASPP,
        'channels': 4,
        'pyramid': (2, 4, 8), 
        'input_type': 'list',
        'output_type': 'list',
    },
    'decoder': {
        'type': 'decoder',
        'num_stages': 2,
        'blocks': {'base_block': ResBlock, 'channels': 'same'},
        'upsample': {'layout': 'b', 'factor': 2}
    },
    'head': {
        'layout': 'c',
        'channels': 1
    }
}

ppl = run('segmentation', TorchModel, config, 'unet-like with ASPP', n_iters=100, batch_size=128)

# Segmentation: named networks

In [ ]:
config = {
    'initial_block': {'layout': 'cna', 'channels': 4},
    'encoder': {'blocks/channels': 'int(same * 1.3)'},
    'embedding': {'channels': 'same'},
    'decoder': {'blocks/channels': 'int(same // 1.3)',
                'upsample': {'layout': 'b', 'factor': 2}},
    'head': {'layout': 'c', 'channels': 1}
}

ppl = run('segmentation', UNet, config, 'unet')

In [ ]:
ppl = run('segmentation', ResUNet, config, 'unet with residual blocks')

In [ ]:
config = {
    'initial_block': {'layout': 'cna', 'channels': 4},
    'encoder/num_stages': 2,
    'embedding/channels': 6,
    'decoder/blocks/channels': 6,
}

ppl = run('segmentation', DenseUNet, config, 'unet with dense blocks')

# Segmentation: imported encoders

In [ ]:
config = {
    'order': ['initial_block', 'encoder', 'decoder', 'head'],
    
    'initial_block': {
        'layout': 'cna',
        'channels': 3,
        'output_type': 'list',
    },
    'encoder': {
        'type': 'hugging-face',
        'path': 'nvidia/segformer-b0-finetuned-ade-512-512',
        'num_encoder_blocks': 2,                              # ~num_stages
        'strides': [2, 2],
        'num_attention_heads': [1, 4],
        'hidden_sizes': [48, 64],                             # ~channels
    },
    'decoder': {
        'type': 'mlpdecoder',
        'upsample/features': 16,
        'block/channels': 16,
    },

    'head': {'layout': 'c',
             'channels': 1},
}

ppl = run('segmentation', TorchModel, config, 'HF-SegFormer + mlp-decoder', n_iters=100, batch_size=128)
ppl.model.repr(1)

In [ ]:
ppl.model.model.repr(2, show_num_parameters=True)

In [ ]:
config = {
    'order': ['initial_block', 'encoder', 'embedding', 'decoder', 'head'],
    
    'initial_block': {
        'layout': 'cna',
        'channels': 3,
        'output_type': 'list',
    },
    'encoder': {
        'type': 'timm',
        'path': 'resnet18',
        'pretrained': True,
        'out_indices': (0, 1, 2),
    },
    'embedding': {
        'base_block': ResBlock,
        'channels': 'same',
        'input_type': 'list',
        'output_type': 'list',
    },
    'decoder': {
        'type': 'decoder',
        'num_stages': 3,
        'blocks': {'base_block': ResBlock, 'channels': 'same // 4'},
        'upsample': {'layout': 'b', 'factor': 2},
    },

    'head': {'layout': 'c',
             'channels': 1},
}

ppl = run('segmentation', TorchModel, config, 'TIMM-ResNet18 + decoder', n_iters=100, batch_size=128)
ppl.model.repr(1)

In [ ]:
ppl.model.model.repr(2, show_num_parameters=True)

# Callbacks

In [ ]:
config = {
    'callbacks': [
    ReduceLROnPlateau(patience=20, cooldown=20, min_delta=0.1, factor=0.9),
    EarlyStopping(patience=100, min_delta=0.01)
    ],
    'decay': {'name': 'exp', 'gamma': 0.8, 'frequency': 20},
}

with Monitor() as monitor:
    ppl = run('classification', ResNet34, config, 'resnet34', batch_size=64, n_iters=200)
    
test(ppl, show_results=False);

# Model info

In [ ]:
ppl.model.show_loss()
# ppl.model.show_lr() # only learning rate

In [ ]:
monitor.plot()

In [ ]:
ppl.model.information()

# AMP

In [ ]:
config = {
    'amp': False,
}

ppl = run('classification', ResNet34, config, 'resnet34', batch_size=64, n_iters=200)

In [ ]:
config = {
    'amp': True # default
}

ppl = run('classification', ResNet34, config, 'resnet34', batch_size=64, n_iters=200)

In [ ]:
config = {
    'amp': True,
    'sam_rho': 0.05,
}

ppl = run('classification', ResNet34, config, 'resnet34', batch_size=64, n_iters=200)

# Wrappers: TTA + augmentations on train

In [ ]:
from torchvision import transforms

config = {
    'order': ['aug0', 'preprocess', 'initial_block', 'body', 'head'],
    
    'aug0': {
        'module': transforms.Compose([
            # transforms.RandomAdjustSharpness(0.1, p=1.0),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomHorizontalFlip(p=0.5),
        ]),
        'disable_at_inference': True,
    },
    'preprocess': {
        'module': transforms.Resize(32),
        'disable_at_inference': False,
    },
}


ppl = run('classification', ResNet18, config, 'resnet18', n_iters=100, batch_size=128)
print(f'Used modules at train: {ppl.model.model._last_used_modules}')

test(ppl, show_results=False)
print(f'Used modules at inference: {ppl.model.model._last_used_modules}')

In [ ]:
ppl.model.repr(2)

In [ ]:
import ttach
ppl.model.wrap_tta(wrapper='ClassificationTTAWrapper',
                   transforms=ttach.aliases.vlip_transform(),
                   merge_mode='mean')

test(ppl, show_results=False)
print(f'Used modules at inference: {ppl.model.model.model._last_used_modules}')

# Wrappers: TRT demo

In [ ]:
BATCH_SIZE = 128

In [ ]:
ppl = run('classification', ResNet34, {}, 'resnet34', batch_size=64, n_iters=200)
test(ppl, show_results=False, batch_size=BATCH_SIZE);

model = ppl.model

In [ ]:
%%time
from torch2trt import torch2trt

module = model.model.eval()
inputs = model.make_placeholder_data(batch_size=BATCH_SIZE, unwrap=False)

model_trt = torch2trt(module=module.eval(), inputs=inputs,
                      fp16_mode=True, use_onnx=True, max_batch_size=BATCH_SIZE)

In [ ]:
diff = torch.abs(model_trt(*inputs) - module.eval()(*inputs))
torch.max(diff), torch.mean(diff)

In [ ]:
%%timeit -r 20 -n 100
inputs = model.make_placeholder_data(batch_size=BATCH_SIZE)

with torch.no_grad():
    output = module.eval()(inputs).cpu()
torch.cuda.synchronize()

del output, inputs
torch.cuda.empty_cache()

In [ ]:
%%timeit -r 20 -n 100
inputs = model.make_placeholder_data(batch_size=BATCH_SIZE)

with torch.no_grad():
    output = model_trt(inputs).cpu()
torch.cuda.synchronize()

del output, inputs
torch.cuda.empty_cache()

# Wrappers: TRT

In [ ]:
%%time
model.wrap_trt(batch_size=BATCH_SIZE)

In [ ]:
%%timeit -r 20 -n 100
inputs = model.make_placeholder_data(batch_size=BATCH_SIZE)

with torch.no_grad():
    output = model.model(inputs).cpu()
torch.cuda.synchronize()

del output, inputs
torch.cuda.empty_cache()

In [ ]:
inputs = model.make_placeholder_data(batch_size=1)

In [ ]:
model.model(inputs)

In [ ]:
model.model(torch.tile(inputs, (BATCH_SIZE, 1, 1, 1)))

In [ ]:
test(ppl, show_results=False, batch_size=BATCH_SIZE, drop_last=True);

# Wrappers: TTA + TRT

In [ ]:
ppl = run('classification', ResNet34, {}, 'resnet34', batch_size=BATCH_SIZE, n_iters=200)

model = ppl.model

In [ ]:
%%time
test(ppl, show_results=False, batch_size=BATCH_SIZE, drop_last=True, n_epochs=5);

In [ ]:
%%time
model.wrap_trt(batch_size=BATCH_SIZE, fp16_mode=True)

# import ttach
# ppl.model.wrap_tta(wrapper='ClassificationTTAWrapper',
#                    transforms=ttach.aliases.vlip_transform(),
#                    merge_mode='mean')

In [ ]:
%%time
test(ppl, show_results=False, batch_size=BATCH_SIZE, drop_last=True, n_epochs=5);